# Instância 1 - freeloading 1

In [12]:
import gurobipy as gp
from gurobipy import GRB

def escrever_solucao_em_arquivo(model, p, x, y, z, boxes_dims, L, W, H, tipo_dict, filename):
    """
    Escreve o layout da solução final em um arquivo de texto.

    Formato do Arquivo:
    - Linha 1: L W H
    - Linhas seguintes: x y z l w h tipo cliente (para cada caixa empacotada)
    """
    print(f"\n--- Escrevendo solução no arquivo '{filename}'... ---")
    try:
        with open(filename, 'w') as f:
            # Escreve a primeira linha com as dimensões do contêiner
            f.write(f"{L} {W} {H}\n")
            
            # Itera por todas as caixas para verificar quais foram empacotadas
            m = len(boxes_dims)
            for i in range(m):
                # Apenas escreve no arquivo se a caixa foi de fato incluída na solução
                if p[i].X > 0.5:
                    li, wi, hi = boxes_dims[i]
                    x_i, y_i, z_i = x[i].X, y[i].X, z[i].X
                    tipo = tipo_dict[boxes_dims[i]]
                    cliente = 1  # Cliente fixo como 1, conforme solicitado
                    
                    # Escreve a linha formatada para a caixa
                    f.write(f"{x_i:.4f} {y_i:.4f} {z_i:.4f} {li} {wi} {hi} {tipo} {cliente}\n")
        
        print("Arquivo de solução salvo com sucesso.")
    except Exception as e:
        print(f"ERRO: Não foi possível escrever no arquivo de solução. Causa: {e}")


def executar_e_relatar(model, p, x, y, z, boxes_dims, L, W, H, tipo_dict, time_limit_seconds, output_filename):
    """
    Executa um modelo Gurobi, relata os resultados e salva a solução em um arquivo.
    """
    model.setParam('TimeLimit', time_limit_seconds)
    model.setParam('OutputFlag', 0)

    print(f"--- Executando modelo com limite de tempo de {time_limit_seconds} segundos... ---")
    
    model.optimize()

    print("\n" + "="*45)
    print("      RELATÓRIO FINAL DE EXECUÇÃO")
    print("="*45)

    if model.SolCount > 0:
        status_solucao = "Ótima" if model.status == GRB.OPTIMAL else "Subótima (Limite de Tempo Atingido)"
        
        print(f"Status da Solução: {status_solucao}")
        print(f"Valor Final Atingido (FO): {model.ObjVal:.4f}")
        print(f"Gap de Otimalidade: {model.MIPGap * 100:.4f}%")
        print(f"Tempo Computacional Despendido: {model.Runtime:.4f} segundos")
        
        # Chama a função para escrever a solução em um arquivo
        escrever_solucao_em_arquivo(model, p, x, y, z, boxes_dims, L, W, H, tipo_dict, output_filename)
    else:
        print("Status da Solução: Nenhuma solução viável foi encontrada.")
        print(f"Tempo Computacional Despendido: {model.Runtime:.4f} segundos")

    print("="*45 + "\n")




In [4]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 100, 100, 100
M = 1000  # Número grande o suficiente

# Dados das caixas (li, wi, hi)
boxes = [
    (63, 64, 39), (63, 64, 39),         
    (30, 27, 29), (30, 27, 29),  
    (53, 46, 58), (53, 46, 58),
    (62, 52, 42),  
    (58, 54, 73)
]
m = len(boxes)

##################################
#         VOLUME RELATIVO       #
##################################
vi = [l*w*h/(L*W*H) for l,w,h in boxes]

model = gp.Model("container_absolute")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# Restrições
# 1. Caixas dentro do container (eliminamos Xo, Yo, Zo)
for i in range(m):
    li, wi, hi = boxes[i]
    # Caixa dentro do container se p_i=1
    # obs se p_1 = 0, a desigualdade é sempre verdadeira pois M é um número grande
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
    # Caixa automaticamente na posição (0, 0, 0) se p_i=0
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # 2. Não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# 3. Restrição para lidar com a simetria de caixas (por exemplo, se um problema tem 2 caixas 
# do mesmo tipo, a simples troca entre essas duas caixas não deveriam ser consideradas 
# soluções diferentes - e não queremos perder tempo achando elas)
# C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                 # Se duas caixas iguais estiverem empacotadas, colocar uma ordem arbitrária
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]))


tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1
# Rodar o modelo
executar_e_relatar(
    model=model,
    p=p, x=x, y=y, z=z,
    boxes_dims=boxes,
    L=L, W=W, H=H,
    tipo_dict=tipo_dict,
    time_limit_seconds=3600,
    output_filename="freeloading_instancia_1.txt"
)


Set parameter TimeLimit to value 3600
--- Executando modelo com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO
Status da Solução: Ótima
Valor Final Atingido (FO): 0.4870
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0551 segundos

--- Escrevendo solução no arquivo 'freeloading_instancia_1.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 2

In [9]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 100, 100, 100
M = 1000  # Número grande o suficiente


# Dados das caixas (li, wi, hi)
boxes = [
    (43, 57, 70),       
    (59, 45, 62),   
    (57, 65, 46), (57, 65, 46), 
    (27, 27, 62), (27, 27, 62), (27, 27, 62), (27, 27, 62), 
    (38, 38, 39), (38, 38, 39), (38, 38, 39), (38, 38, 39), (38, 38, 39), (38, 38, 39), (38, 38, 39)
]
m = len(boxes)

##################################
#         VOLUME RELATIVO       #
##################################
vi = [l*w*h/(L*W*H) for l,w,h in boxes]

model = gp.Model("container_absolute")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# Restrições
# 1. Caixas dentro do container (eliminamos Xo, Yo, Zo)
for i in range(m):
    li, wi, hi = boxes[i]
    # Caixa dentro do container se p_i=1
    # obs se p_1 = 0, a desigualdade é sempre verdadeira pois M é um número grande
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
    # Caixa automaticamente na posição (0, 0, 0) se p_i=0
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # 2. Não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# 3. Restrição para lidar com a simetria de caixas (por exemplo, se um problema tem 2 caixas 
# do mesmo tipo, a simples troca entre essas duas caixas não deveriam ser consideradas 
# soluções diferentes - e não queremos perder tempo achando elas)
# C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                 # Se duas caixas iguais estiverem empacotadas, colocar uma ordem arbitrária
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]))



tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1
        
# Rodar o modelo
executar_e_relatar(
    model=model,
    p=p, x=x, y=y, z=z,
    boxes_dims=boxes,
    L=L, W=W, H=H,
    tipo_dict=tipo_dict,
    time_limit_seconds=3600,
    output_filename="freeloading_instancia_2.txt"
)


Set parameter TimeLimit to value 3600
--- Executando modelo com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO
Status da Solução: Ótima
Valor Final Atingido (FO): 0.7155
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 32.7328 segundos

--- Escrevendo solução no arquivo 'freeloading_instancia_2.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 3

In [11]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 100, 100, 100
M = 1000  # Número grande o suficiente


# Dados das caixas (li, wi, hi)
boxes = [
    (36, 52, 33 ), (36, 52, 33 ), (36, 52, 33 ), (36, 52, 33 ), (36, 52, 33 ),   
    (53, 27, 30),  (53, 27, 30), (53, 27, 30), (53, 27, 30),  
    (45, 29, 62), (45, 29, 62), (45, 29, 62),
    (40, 60, 65), (40, 60, 65),
    ( 65, 51, 43),  ( 65, 51, 43)
]
m = len(boxes)

##################################
#         VOLUME RELATIVO       #
##################################
vi = [l*w*h/(L*W*H) for l,w,h in boxes]

model = gp.Model("container_absolute")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# Restrições
# 1. Caixas dentro do container (eliminamos Xo, Yo, Zo)
for i in range(m):
    li, wi, hi = boxes[i]
    # Caixa dentro do container se p_i=1
    # obs se p_1 = 0, a desigualdade é sempre verdadeira pois M é um número grande
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
    # Caixa automaticamente na posição (0, 0, 0) se p_i=0
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # 2. Não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# 3. Restrição para lidar com a simetria de caixas (por exemplo, se um problema tem 2 caixas 
# do mesmo tipo, a simples troca entre essas duas caixas não deveriam ser consideradas 
# soluções diferentes - e não queremos perder tempo achando elas)
# C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                 # Se duas caixas iguais estiverem empacotadas, colocar uma ordem arbitrária
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]))



tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1
        
# Rodar o modelo
executar_e_relatar(
    model=model,
    p=p, x=x, y=y, z=z,
    boxes_dims=boxes,
    L=L, W=W, H=H,
    tipo_dict=tipo_dict,
    time_limit_seconds=3600,
    output_filename="freeloading_instancia_3.txt"
)


Set parameter TimeLimit to value 3600
--- Executando modelo com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO
Status da Solução: Ótima
Valor Final Atingido (FO): 0.7073
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 266.2529 segundos

--- Escrevendo solução no arquivo 'freeloading_instancia_3.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 4

In [13]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 100, 100, 100
M = 1000  # Número grande o suficiente


# Dados das caixas (li, wi, hi)
boxes = [
    (55, 71, 68 ),    
    (62, 71, 51 ),  
    (31, 26, 66 ),(31, 26, 66 ),(31, 26, 66 ),(31, 26, 66 ),(31, 26, 66 ),(31, 26, 66 ),(31, 26, 66 ),(31, 26, 66 ),(31, 26, 66 ),
    (28, 56, 66  ), (28, 56, 66  ), 
    (61, 68, 50 )
]
m = len(boxes)

##################################
#         VOLUME RELATIVO       #
##################################
vi = [l*w*h/(L*W*H) for l,w,h in boxes]

model = gp.Model("container_absolute")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# Restrições
# 1. Caixas dentro do container (eliminamos Xo, Yo, Zo)
for i in range(m):
    li, wi, hi = boxes[i]
    # Caixa dentro do container se p_i=1
    # obs se p_1 = 0, a desigualdade é sempre verdadeira pois M é um número grande
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
    # Caixa automaticamente na posição (0, 0, 0) se p_i=0
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # 2. Não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# 3. Restrição para lidar com a simetria de caixas (por exemplo, se um problema tem 2 caixas 
# do mesmo tipo, a simples troca entre essas duas caixas não deveriam ser consideradas 
# soluções diferentes - e não queremos perder tempo achando elas)
# C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                 # Se duas caixas iguais estiverem empacotadas, colocar uma ordem arbitrária
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]))



tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1
        
# Rodar o modelo
executar_e_relatar(
    model=model,
    p=p, x=x, y=y, z=z,
    boxes_dims=boxes,
    L=L, W=W, H=H,
    tipo_dict=tipo_dict,
    time_limit_seconds=3600,
    output_filename="freeloading_instancia_4.txt"
)


Set parameter TimeLimit to value 3600
--- Executando modelo com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO
Status da Solução: Ótima
Valor Final Atingido (FO): 0.5315
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 43.5518 segundos

--- Escrevendo solução no arquivo 'freeloading_instancia_4.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 5

In [15]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 100, 100, 100
M = 1000  # Número grande o suficiente


# Dados das caixas (li, wi, hi)
boxes = [
    (67, 29, 40),    (67, 29, 40),  (67, 29, 40),  (67, 29, 40),  (67, 29, 40),  (67, 29, 40),  
    (70, 42, 36),  (70, 42, 36),  (70, 42, 36),  
    (64, 48, 68),
    (37, 55, 56), 
    (30, 71, 45), (30, 71, 45), (30, 71, 45), (30, 71, 45), (30, 71, 45)
]
m = len(boxes)

##################################
#         VOLUME RELATIVO       #
##################################
vi = [l*w*h/(L*W*H) for l,w,h in boxes]

model = gp.Model("container_absolute")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# Restrições
# 1. Caixas dentro do container (eliminamos Xo, Yo, Zo)
for i in range(m):
    li, wi, hi = boxes[i]
    # Caixa dentro do container se p_i=1
    # obs se p_1 = 0, a desigualdade é sempre verdadeira pois M é um número grande
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
    # Caixa automaticamente na posição (0, 0, 0) se p_i=0
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # 2. Não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# 3. Restrição para lidar com a simetria de caixas (por exemplo, se um problema tem 2 caixas 
# do mesmo tipo, a simples troca entre essas duas caixas não deveriam ser consideradas 
# soluções diferentes - e não queremos perder tempo achando elas)
# C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                 # Se duas caixas iguais estiverem empacotadas, colocar uma ordem arbitrária
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]))



tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1
        
# Rodar o modelo
executar_e_relatar(
    model=model,
    p=p, x=x, y=y, z=z,
    boxes_dims=boxes,
    L=L, W=W, H=H,
    tipo_dict=tipo_dict,
    time_limit_seconds=3600,
    output_filename="freeloading_instancia_5.txt"
)


Set parameter TimeLimit to value 3600
--- Executando modelo com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO
Status da Solução: Ótima
Valor Final Atingido (FO): 0.7405
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 3428.5341 segundos

--- Escrevendo solução no arquivo 'freeloading_instancia_5.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 6

In [17]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 100, 100, 100
M = 1000  # Número grande o suficiente


# Dados das caixas (li, wi, hi)
boxes = [
    (72, 25, 26), (72, 25, 26), (72, 25, 26), (72, 25, 26), (72, 25, 26), (72, 25, 26), (72, 25, 26),
    (43, 38, 71) ,
    (64, 65, 74),
    (54, 61, 60), 
    (27, 55, 52)
]
m = len(boxes)

##################################
#         VOLUME RELATIVO       #
##################################
vi = [l*w*h/(L*W*H) for l,w,h in boxes]

model = gp.Model("container_absolute")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# Restrições
# 1. Caixas dentro do container (eliminamos Xo, Yo, Zo)
for i in range(m):
    li, wi, hi = boxes[i]
    # Caixa dentro do container se p_i=1
    # obs se p_1 = 0, a desigualdade é sempre verdadeira pois M é um número grande
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
    # Caixa automaticamente na posição (0, 0, 0) se p_i=0
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # 2. Não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# 3. Restrição para lidar com a simetria de caixas (por exemplo, se um problema tem 2 caixas 
# do mesmo tipo, a simples troca entre essas duas caixas não deveriam ser consideradas 
# soluções diferentes - e não queremos perder tempo achando elas)
# C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                 # Se duas caixas iguais estiverem empacotadas, colocar uma ordem arbitrária
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]))



tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1
        
# Rodar o modelo
executar_e_relatar(
    model=model,
    p=p, x=x, y=y, z=z,
    boxes_dims=boxes,
    L=L, W=W, H=H,
    tipo_dict=tipo_dict,
    time_limit_seconds=3600,
    output_filename="freeloading_instancia_6.txt"
)


Set parameter TimeLimit to value 3600
--- Executando modelo com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO
Status da Solução: Ótima
Valor Final Atingido (FO): 0.6717
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 7.8665 segundos

--- Escrevendo solução no arquivo 'freeloading_instancia_6.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 7

In [18]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 100, 100, 100
M = 1000  # Número grande o suficiente


# Dados das caixas (li, wi, hi)
boxes = [
    (60, 40, 58), (60, 40, 58),
    (71, 39, 43), (71, 39, 43), (71, 39, 43),
    (68, 69, 54),
    (38, 53, 53), 
    (27, 72, 54), (27, 72, 54)
]
m = len(boxes)

##################################
#         VOLUME RELATIVO       #
##################################
vi = [l*w*h/(L*W*H) for l,w,h in boxes]

model = gp.Model("container_absolute")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# Restrições
# 1. Caixas dentro do container (eliminamos Xo, Yo, Zo)
for i in range(m):
    li, wi, hi = boxes[i]
    # Caixa dentro do container se p_i=1
    # obs se p_1 = 0, a desigualdade é sempre verdadeira pois M é um número grande
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
    # Caixa automaticamente na posição (0, 0, 0) se p_i=0
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # 2. Não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# 3. Restrição para lidar com a simetria de caixas (por exemplo, se um problema tem 2 caixas 
# do mesmo tipo, a simples troca entre essas duas caixas não deveriam ser consideradas 
# soluções diferentes - e não queremos perder tempo achando elas)
# C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                 # Se duas caixas iguais estiverem empacotadas, colocar uma ordem arbitrária
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]))



tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1
        
# Rodar o modelo
executar_e_relatar(
    model=model,
    p=p, x=x, y=y, z=z,
    boxes_dims=boxes,
    L=L, W=W, H=H,
    tipo_dict=tipo_dict,
    time_limit_seconds=3600,
    output_filename="freeloading_instancia_7.txt"
)


Set parameter TimeLimit to value 3600
--- Executando modelo com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO
Status da Solução: Ótima
Valor Final Atingido (FO): 0.6739
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0917 segundos

--- Escrevendo solução no arquivo 'freeloading_instancia_7.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 8

In [20]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 100, 100, 100
M = 1000  # Número grande o suficiente


# Dados das caixas (li, wi, hi)
boxes = [
    (27, 50, 46), (27, 50, 46), (27, 50, 46), (27, 50, 46), (27, 50, 46), (27, 50, 46),
    (65, 62, 43 ),
    (45, 74, 55),
    (48, 60, 69), 
    (49, 51, 34), (49, 51, 34), (49, 51, 34)
]
m = len(boxes)

##################################
#         VOLUME RELATIVO       #
##################################
vi = [l*w*h/(L*W*H) for l,w,h in boxes]

model = gp.Model("container_absolute")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# Restrições
# 1. Caixas dentro do container (eliminamos Xo, Yo, Zo)
for i in range(m):
    li, wi, hi = boxes[i]
    # Caixa dentro do container se p_i=1
    # obs se p_1 = 0, a desigualdade é sempre verdadeira pois M é um número grande
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
    # Caixa automaticamente na posição (0, 0, 0) se p_i=0
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # 2. Não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# 3. Restrição para lidar com a simetria de caixas (por exemplo, se um problema tem 2 caixas 
# do mesmo tipo, a simples troca entre essas duas caixas não deveriam ser consideradas 
# soluções diferentes - e não queremos perder tempo achando elas)
# C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                 # Se duas caixas iguais estiverem empacotadas, colocar uma ordem arbitrária
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]))



tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1
        
# Rodar o modelo
executar_e_relatar(
    model=model,
    p=p, x=x, y=y, z=z,
    boxes_dims=boxes,
    L=L, W=W, H=H,
    tipo_dict=tipo_dict,
    time_limit_seconds=3600,
    output_filename="freeloading_instancia_8.txt"
)


Set parameter TimeLimit to value 3600
--- Executando modelo com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO
Status da Solução: Ótima
Valor Final Atingido (FO): 0.7290
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.2514 segundos

--- Escrevendo solução no arquivo 'freeloading_instancia_8.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 9

In [21]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 100, 100, 100
M = 1000  # Número grande o suficiente


# Dados das caixas (li, wi, hi)
boxes = [
    (36, 38, 59), (36, 38, 59), (36, 38, 59),
    (30, 55, 25), (30, 55, 25), (30, 55, 25), (30, 55, 25), 
    (69, 59, 69),
    (72, 47, 28),  (72, 47, 28),  (72, 47, 28),  (72, 47, 28),
    (36, 55, 45), (36, 55, 45)
]
m = len(boxes)

##################################
#         VOLUME RELATIVO       #
##################################
vi = [l*w*h/(L*W*H) for l,w,h in boxes]

model = gp.Model("container_absolute")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# Restrições
# 1. Caixas dentro do container (eliminamos Xo, Yo, Zo)
for i in range(m):
    li, wi, hi = boxes[i]
    # Caixa dentro do container se p_i=1
    # obs se p_1 = 0, a desigualdade é sempre verdadeira pois M é um número grande
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
    # Caixa automaticamente na posição (0, 0, 0) se p_i=0
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # 2. Não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# 3. Restrição para lidar com a simetria de caixas (por exemplo, se um problema tem 2 caixas 
# do mesmo tipo, a simples troca entre essas duas caixas não deveriam ser consideradas 
# soluções diferentes - e não queremos perder tempo achando elas)
# C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                 # Se duas caixas iguais estiverem empacotadas, colocar uma ordem arbitrária
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]))



tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1
        
# Rodar o modelo
executar_e_relatar(
    model=model,
    p=p, x=x, y=y, z=z,
    boxes_dims=boxes,
    L=L, W=W, H=H,
    tipo_dict=tipo_dict,
    time_limit_seconds=3600,
    output_filename="freeloading_instancia_9.txt"
)


Set parameter TimeLimit to value 3600
--- Executando modelo com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO
Status da Solução: Ótima
Valor Final Atingido (FO): 0.7143
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.5278 segundos

--- Escrevendo solução no arquivo 'freeloading_instancia_9.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 10

In [22]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 100, 100, 100
M = 1000  # Número grande o suficiente


# Dados das caixas (li, wi, hi)
boxes = [
    (46, 33, 73), (46, 33, 73), (46, 33, 73), (46, 33, 73), (46, 33, 73), (46, 33, 73),
    (63, 72, 68),
    (41, 29, 26), (41, 29, 26), (41, 29, 26), (41, 29, 26), (41, 29, 26), (41, 29, 26), (41, 29, 26), (41, 29, 26), (41, 29, 26), 
    (61, 31, 30), (61, 31, 30), (61, 31, 30), (61, 31, 30),
    (27, 48, 55)
]
m = len(boxes)

##################################
#         VOLUME RELATIVO       #
##################################
vi = [l*w*h/(L*W*H) for l,w,h in boxes]

model = gp.Model("container_absolute")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# Restrições
# 1. Caixas dentro do container (eliminamos Xo, Yo, Zo)
for i in range(m):
    li, wi, hi = boxes[i]
    # Caixa dentro do container se p_i=1
    # obs se p_1 = 0, a desigualdade é sempre verdadeira pois M é um número grande
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
    # Caixa automaticamente na posição (0, 0, 0) se p_i=0
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # 2. Não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# 3. Restrição para lidar com a simetria de caixas (por exemplo, se um problema tem 2 caixas 
# do mesmo tipo, a simples troca entre essas duas caixas não deveriam ser consideradas 
# soluções diferentes - e não queremos perder tempo achando elas)
# C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                 # Se duas caixas iguais estiverem empacotadas, colocar uma ordem arbitrária
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]))



tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1
        
# Rodar o modelo
executar_e_relatar(
    model=model,
    p=p, x=x, y=y, z=z,
    boxes_dims=boxes,
    L=L, W=W, H=H,
    tipo_dict=tipo_dict,
    time_limit_seconds=3600,
    output_filename="freeloading_instancia_10.txt"
)


Set parameter TimeLimit to value 3600
--- Executando modelo com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO
Status da Solução: Subótima (Limite de Tempo Atingido)
Valor Final Atingido (FO): 0.8504
Gap de Otimalidade: 10.9061%
Tempo Computacional Despendido: 3600.0382 segundos

--- Escrevendo solução no arquivo 'freeloading_instancia_10.txt'... ---
Arquivo de solução salvo com sucesso.

